# Kalman filtering and Smoothing for the river nile data

This notebook analyzes the River Nile data using the local level model with full implementation of the Kalman filter and smoother.

In [6]:
using Pkg
Pkg.add("Plots");

   Resolving package versions...
  No Changes to `~/Dropbox/BayesBookWeb/BayesianLearningBook/notebooks/KalmanFilteringSmoothing/Project.toml`
  No Changes to `~/Dropbox/BayesBookWeb/BayesianLearningBook/notebooks/KalmanFilteringSmoothing/Manifest.toml`


Define the Kalman filter update at a given time point:

In [7]:
function kalmanfilter_update(μ, Ω, u, y, A, B, C, Σₑ, Σₙ)

    # Prediction step - moving state forward without new measurement
    μ̄ = A*μ .+ B*u
    Ω̄ = A*Ω*A' + Σₙ

    # Measurement update - updating the N(μ̄, Ω̄) prior with the new data point
    K = Ω̄*C' / (C*Ω̄*C' .+ Σₑ) # Kalman Gain
    μ = μ̄ + K*(y .- C*μ̄)
    Ω = (I - K*C)*Ω̄
    return μ, Ω

end;

Defining a function for the Kalman filter algorithm that calls on the *kalmanfilter_update* at each time-step:

In [8]:
function kalmanfilter(U, Y, A, B, C, Σₑ, Σₙ, μ₀, Σ₀)
    
    T = size(Y,1)   # Number of time steps
    n = length(μ₀)  # Dimension of the state vector      

    # Storage for the mean and covariance state vector trajectory over time
    μ_filter = zeros(T, n)   
    Σ_filter = zeros(n, n, T)
    
    # The Kalman iterations
    μ = μ₀
    Σ = Σ₀
    for t = 1:T
        μ, Σ = kalmanfilter_update(μ, Σ, U[t,:]', Y[t,:]', A, B, C, Σₑ, Σₙ)
        μ_filter[t,:] .= μ
        Σ_filter[:,:,t] .= Σ
    end

    return μ_filter, Σ_filter

end;